# Deep learning con Python
## Redes Neuronales Convolucionales (CNN)

En esta parte del taller, vamos a aprender a usar CNNs para reconocer objetos en imágenes urbanas (vehículos, árboles, etc.).

Este notebook incluye:

- Explicación introducctoria.
- Preparación de dataset (COCO),
- Dos modelos CNN (sencillo y más complejo).
- Uso de una CNN ya pre-entrenada (YOLO, Ultralytics).
- Visualización de las imágenes.
- Ejercicios.

---

## 0) ¿Qué es una CNN?

- Capas convolucionales (filtros) que detectan patrones locales.
- Pooling para reducir resolución y aportar invariancia local.
- BatchNorm/Dropout para estabilizar/regularizar.

**Tareas:** clasificación de imagen, detección (bounding boxes), segmentación (pixel-wise). En este taller haremos clasificación mediante crops y detección con YOLO.

In [ ]:
from IPython.display import Image, display

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_cnn1.png",width=800, height=300))

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_conv_operator.png",width=800, height=400))

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_pooling.gif",width=800, height=600))

¿Y que se entiende por segmentación de imágenes?

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_image_segmentation.png",width=1000, height=600))

Para ello vamos a seguir el mismo flujo de trabajo que ya usamos a la hora de desarrollar nuestras MLP y RNN.



In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_pipeline.jpg",width=800, height=300))

---

## 1) Librerías Principales


In [ ]:
import os
import json
import random
from pprint import pprint
from PIL import Image as im
from PIL import ImageDraw
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# reproducibilidad
RND = 42
np.random.seed(RND)

---

## 2) Obtener el dataset de imágenes sobre el que vamos a trabajar (COCO)


COCO (*Common Objects in Context*) es un dataset a gran escala para detección de objetos, segmentación y captioning. Contiene cientos de miles de imágenes y millones de instancias anotadas.

COCO distribuye particiones (train/val/test) con anotaciones detalladas (bounding boxes, segmentaciones por instancia, categorías, keypoints para personas en algunos splits). Las anotaciones para train y val vienen en ficheros JSON como `instances_train2017.json` y `instances_val2017.json`.
COCO Dataset

En cuanto a su formato COCO es un único JSON con claves principales images, annotations, categories (entre otras). Cada annotation tiene campos como `bbox` (x,y,w,h), `segmentation` (si aplica), `category_id`, `area`, `iscrowd`, `id`.

A continuación vamos a descargar dicho dataset y sus anotaciones.

In [ ]:
DATA_DIR = 'data'
IMAGES_DIR = os.path.join(DATA_DIR, 'val2017')          # carpeta con imágenes val2017
ANN_FILE = os.path.join(DATA_DIR, 'annotations', 'instances_val2017.json')

In [ ]:
os.makedirs(DATA_DIR, exist_ok=True)
!wget -nc -P {DATA_DIR} http://images.cocodataset.org/zips/val2017.zip
!wget -nc -P {DATA_DIR} http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -n {DATA_DIR}/val2017.zip -d {DATA_DIR}
!unzip -n {DATA_DIR}/annotations_trainval2017.zip -d {DATA_DIR}

Vamos a visualizar el formado del dataset que hemos obtenido

In [ ]:
with open(ANN_FILE, 'r', encoding='utf-8') as f:
    coco = json.load(f)

    # Información resumida
    n_images = len(coco.get('images', []))
    n_anns = len(coco.get('annotations', []))
    n_cats = len(coco.get('categories', []))
    print(f"Resumen COCO (archivo {os.path.basename(ANN_FILE)}):")
    print(f" - imágenes (registro JSON): {n_images}")
    print(f" - anotaciones: {n_anns}")
    print(f" - categorías: {n_cats}")
    print()

    # Mostrar las primeras categorías (id:name)
    cat_map = {c['id']: c['name'] for c in coco.get('categories', [])}
    print("Algunas categorías (id -> name):")
    for cid in sorted(list(cat_map.keys()))[:20]:
        print(f"  {cid} -> {cat_map[cid]}")


Vamos ahora a mostrar una imágen y los objetos en ella contenidos.

In [ ]:
img_rec = None
ann_for_img= None

with open(ANN_FILE, 'r', encoding='utf-8') as f:
  coco = json.load(f)
  img_rec = random.choice(coco['images'])
  img_id = img_rec['id']
  img_filename = img_rec.get('file_name', 'N/A')
  img_path = os.path.join(IMAGES_DIR, img_filename)

  print("Imagen elegida (registro JSON):")
  pprint(img_rec)
  print()

  if os.path.isfile(img_path):
      # abrir y mostrar la imagen
      img = im.open(img_path).convert('RGB')
      plt.figure(figsize=(8,8))
      plt.imshow(img)
      plt.axis('off')
      plt.title(f"ID {img_id}  —  {img_filename}")
      plt.show()
  else:
      print("No se encontró la imagen en disco:", img_path)
      print("Asegúrate de haber descomprimido val2017.zip en:", IMAGES_DIR)
      # no terminamos; aún mostramos las anotaciones (si existen en JSON)

  # 4) Buscar anotaciones asociadas a esta imagen
  anns_for_img = [a for a in coco['annotations'] if a['image_id'] == img_id]
  print(f"Se encontraron {len(anns_for_img)} anotaciones para la imagen (id={img_id}).\n")

  # Mostrar (y explicar) las claves más importantes de la primera anotación
  if len(anns_for_img) > 0:
      print("Ejemplo de anotación (primera):")
      example_ann = anns_for_img[0]
      # imprimimos los campos más relevantes
      keys_of_interest = ['id','image_id','category_id','bbox','area','iscrowd','segmentation']
      for k in keys_of_interest:
          print(f"  {k} :", example_ann.get(k, None))
      print("\n(El bbox está en formato [x, y, width, height] con coordenadas en píxels.)")

      print("\nTodas las anotaciones (resumen):")
      # imprimimos un resumen compacto de cada anotación: id, catname, bbox
      for a in anns_for_img:
          cid = a['category_id']
          cname = cat_map.get(cid, str(cid))
          bbox = a.get('bbox', None)
          print(f" - ann_id={a['id']:6d}  cat={cname:12s}  bbox={bbox}")
  else:
      print("No hay anotaciones para esta imagen en el JSON.")

Volvamos a imprimir la imagen pero superponiendo los objetos etiquetados en el dataset

In [ ]:
import matplotlib.patches as patches

# Mostrar la imagen con bounding boxes y etiquetas
if os.path.isfile(img_path):
    img = im.open(img_path).convert('RGB')

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"ID {img_id}  —  {img_filename}")

    # Pintar cada anotación
    for a in anns_for_img:
        cid = a['category_id']
        cname = cat_map.get(cid, str(cid))
        bbox = a.get('bbox', None)  # formato [x, y, width, height]

        if bbox:
            x, y, w, h = bbox
            # Añadir rectángulo
            rect = patches.Rectangle(
                (x, y), w, h,
                linewidth=2,
                edgecolor='red',
                facecolor='none'
            )
            ax.add_patch(rect)

            # Etiqueta sobre el bbox
            ax.text(
                x, y - 5, cname,
                fontsize=10,
                color='white',
                bbox=dict(facecolor='red', alpha=0.5, pad=1)
            )

    plt.show()
else:
    print("No se encontró la imagen en disco:", img_path)
    print("Asegúrate de haber descomprimido val2017.zip en:", IMAGES_DIR)


---

## 3) Pre-procesado básico

Vamos a desarrollar una red convolucional que sepa detectar dada una imágen, si hay un vehículo en ella. Por tanto, nos enfrentamos a un problema de clasificacion binaria al iguial que vimos en el notebook de MLPs (coche vs no coche)

Vamos extraer las imágenes de COCO cuyos bboxes sean `car`, `truck` o `bus` para generar ejemplos (`crops`) positivos. Para generar crops negativo, es decir, imágenes que no contengan ninguna de las etiquetas anteriores, vamos a extraer imágenes aleatorias bajo la etiqueta `background`.




In [ ]:
out_dir = os.path.join(DATA_DIR, 'crops')

with open(ANN_FILE, 'r') as f:
  coco = json.load(f)
  cat_map = {c['id']: c['name'] for c in coco['categories']}
  vehicle_cat_ids = [cid for cid, name in cat_map.items() if name in ('car','truck','bus')]

  shutil.rmtree(out_dir, ignore_errors=True)
  os.makedirs(os.path.join(out_dir, 'vehicle'), exist_ok=True)
  os.makedirs(os.path.join(out_dir, 'background'), exist_ok=True)

  images_index = {img['id']: img for img in coco['images']}

  MAX_VEHICLE = 2000
  vehicle_count = 0
  background_count = 0

  for ann in tqdm(coco['annotations'], desc="Extrayendo imágenes con vehículos..."):
      if ann['category_id'] in vehicle_cat_ids and vehicle_count < MAX_VEHICLE:
          img_info = images_index[ann['image_id']]
          img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
          if not os.path.exists(img_path):
              continue
          img = im.open(img_path).convert('RGB')
          x,y,w,h = ann['bbox']
          left = max(0, int(x)); upper = max(0, int(y))
          right = min(img.width, int(x+w)); lower = min(img.height, int(y+h))
          if right-left <= 0 or lower-upper <= 0: continue
          crop = img.crop((left,upper,right,lower)).resize((128,128))
          crop.save(os.path.join(out_dir, 'vehicle', f'veh_{vehicle_count:05d}.jpg'))
          vehicle_count += 1

  random_images = random.sample(list(images_index.values()), min(1000, len(images_index)))
  for img_info in tqdm(random_images, desc="Extrayendo imágenes con background "):
      img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
      if not os.path.exists(img_path): continue
      img = im.open(img_path).convert('RGB')
      for _ in range(3):
          w = random.randint(64, 200)
          h = random.randint(64, 200)
          left = random.randint(0, max(0, img.width - w))
          upper = random.randint(0, max(0, img.height - h))
          crop = img.crop((left,upper,left+w,upper+h)).resize((128,128))
          crop.save(os.path.join(out_dir, 'background', f'bg_{background_count:05d}.jpg'))
          background_count += 1
          if background_count >= vehicle_count and vehicle_count>0: break
      if background_count >= vehicle_count and vehicle_count>0: break

  print('\nImágenes de vehículos', vehicle_count, 'Imágenes background', background_count)

---

## 4) Particionado train/val/test

La clase `ImageDataGenerator` de Keras es una herramienta muy práctica para preprocesar y aumentar (augment) imágenes antes de entrenar una red neuronal convolucional (CNN). Esta clase permite:

- Escala los píxeles a un rango determinado (ej. [0,1] si usas rescale=1./255).

- Permite normalizar, centrar o estandarizar imágenes.

- Aplica transformaciones aleatorias en cada época de entrenamiento (cuando se usa data augmentation).

- Generación de lotes (batches)

- No carga todas las imágenes en memoria.

- Genera batches de imágenes en tiempo real mientras entrenas (fit o fit_generator).

- Es eficiente para trabajar con datasets grandes.

Además permite realizar *Data Augmentation* (aumento de datos) ¿Y eso qué es

- Genera versiones modificadas de las imágenes originales para que el modelo generalice mejor.

Ejemplos de transformaciones:

- Rotaciones (rotation_range)

- Traslaciones horizontales y verticales (width_shift_range, height_shift_range)

- Zoom (zoom_range)

- Volteos horizontales o verticales (horizontal_flip, vertical_flip)

- Brillo o contraste (brightness_range)

- Cortes y recortes aleatorios




In [ ]:
DATA_CROPS = os.path.join('data','crops')
img_size = (128,128)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, horizontal_flip=True)
train_gen = train_datagen.flow_from_directory(DATA_CROPS, target_size=img_size, batch_size=batch_size, subset='training', class_mode='binary')
val_gen = train_datagen.flow_from_directory(DATA_CROPS, target_size=img_size, batch_size=batch_size, subset='validation', class_mode='binary')

---

## 5)  Nuestra primera Red Neuronal Convolucional (CNN) con Keras



In [ ]:
def build_simple_cnn():
    model = models.Sequential([
        layers.Input(shape=(*img_size,3)),
        layers.Conv2D(16, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Vamos a contruir nuestra CNN y ver un resumen de sus parámetros

In [ ]:
simple_model = build_simple_cnn()
simple_model.summary()

In [ ]:
  history_simple = simple_model.fit(train_gen, epochs=5, validation_data=val_gen)

Como ya sabemos mostramos las curvas de aprendizaje

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_simple.history['loss'], label='Entrenamiento')
plt.plot(history_simple.history['val_loss'], label='Validacion')
plt.legend(); plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(history_simple.history['accuracy'], label='Entrenamiento')
plt.plot(history_simple.history['val_accuracy'], label='Validación')
plt.legend(); plt.title('Accuracy')
plt.show()

Vamos a visualizar algunos de los resultados obtenidos por nuestra primera CNN

In [ ]:
crops_veh_dir = os.path.join('data','crops','vehicle')
files = [f for f in os.listdir(crops_veh_dir) if f.lower().endswith('.jpg')]
for f in files[:3]:
    example = os.path.join(crops_veh_dir,f)
    img = im.open(example).convert('RGB').resize((128,128))
    display(img)
    try:
        x = np.array(img)/255.0
        pred = simple_model.predict(x[None,...])[0,0]
        print('Probabilidad de vehiculo:', float(pred))
    except Exception as e:
        print('Modelo  no disponible en memoria. Entrena el modelo antes.')


In [ ]:
crops_bk_dir = os.path.join('data','crops','background')
files = [f for f in os.listdir(crops_bk_dir) if f.lower().endswith('.jpg')]
for f in files[:3]:
    example = os.path.join(crops_bk_dir,f)
    img = im.open(example).convert('RGB').resize((128,128))
    display(img)
    try:
        x = np.array(img)/255.0
        pred = simple_model.predict(x[None,...])[0,0]
        print('Probabilidad de vehiculo:', float(pred))
    except Exception as e:
        print('Modelo no disponible en memoria. Entrena el modelo antes.')

### CNN más compleja



In [ ]:
def build_complex_cnn():
    inp = layers.Input(shape=(*img_size,3))
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
complex_model = build_complex_cnn()
complex_model.summary()

In [ ]:
history_complex = complex_model.fit(train_gen, epochs=5, validation_data=val_gen)

Mostramos sus curvas de aprendizaje

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_complex.history['loss'], label='Entrenamiento')
plt.plot(history_complex.history['val_loss'], label='Validación')
plt.legend(); plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(history_complex.history['accuracy'], label='Entrenamiento')
plt.plot(history_complex.history['val_accuracy'], label='Validación')
plt.legend(); plt.title('Accuracy')
plt.show()

In [ ]:
crops_veh_dir = os.path.join('data','crops','vehicle')
files = [f for f in os.listdir(crops_veh_dir) if f.lower().endswith('.jpg')]
for f in files[:3]:
    example = os.path.join(crops_veh_dir,f)
    img = im.open(example).convert('RGB').resize((128,128))
    display(img)
    try:
        x = np.array(img)/255.0
        pred = complex_model.predict(x[None,...])[0,0]
        print('Probabilidad de vehiculo:', float(pred))
    except Exception as e:
        print('Modelo complejo no disponible en memoria. Entrena el modelo antes.')

In [ ]:
crops_bk_dir = os.path.join('data','crops','background')
files = [f for f in os.listdir(crops_bk_dir) if f.lower().endswith('.jpg')]
for f in files[:3]:
    example = os.path.join(crops_bk_dir,f)
    img = im.open(example).convert('RGB').resize((128,128))
    display(img)
    try:
        x = np.array(img)/255.0
        pred = complex_model.predict(x[None,...])[0,0]
        print('Probabilidad de vehiculo:', float(pred))
    except Exception as e:
        print('Modelo no disponible en memoria. Entrena el modelo antes.')

### Uso de detector pre-entrenado: YOLO

Afortunadamente, existen muchas CNNs pre-entrenadas que podemos usar para analizar los datos.

En nuestro caso vamos a usar, YOLO (You Only Look Once) es una arquitectura de red neuronal convolucional diseñada para detectar objetos en imágenes en tiempo real. A diferencia de métodos clásicos (como R-CNN y sus variantes), YOLO aborda la detección como un único problema de regresión, prediciendo directamente en una sola pasada:

- Las cajas delimitadoras (bounding boxes).

- Las clases de los objetos.

- Las confianzas de cada predicción.

Características principales

- Velocidad: es muy rápido, adecuado para aplicaciones en tiempo real (cámaras de tráfico, vigilancia, coches autónomos).

- Detección múltiple: identifica múltiples objetos en una misma imagen con sus bounding boxes y categorías.

- División en celdas: la imagen se divide en una cuadrícula, y cada celda predice cajas y probabilidades de clase.

- Precisión vs. velocidad: versiones modernas (YOLOv5, YOLOv8) logran un buen equilibrio entre exactitud y rapidez.

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_yolo.jpg",width=800, height=900))

Vamos a instalar la librería que habilita el uso de dicha CNN.

In [ ]:
!pip install ultralytics

Vamos a usar ahora YOLO para detectar objetos, en concreto vehículos sobre el dataset COCO que hemos estado usando.

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
sample_img = None


with open(ANN_FILE, 'r') as f:
  coco = json.load(f)
  cat_map = {c['id']: c['name'] for c in coco['categories']}
  vehicle_cat_ids = [cid for cid, name in cat_map.items() if name in ('car','truck','bus','motorcycle')]

  images_index = {img['id']: img for img in coco['images']}

  vehicle_count=0
  for ann in tqdm(coco['annotations'], desc="Analizando imágenes con vehículos..."):
      if ann['category_id'] in vehicle_cat_ids and vehicle_count < 10:
          img_info = images_index[ann['image_id']]
          img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
          if not os.path.exists(img_path):
              continue
          results = model.predict(source=img_path, conf=0.25)
          img = im.open(img_path).convert('RGB')
          draw = ImageDraw.Draw(img)
          for r in results:
              if hasattr(r, 'boxes') and r.boxes is not None:
                for box in r.boxes:
                    cls_id = int(box.cls[0].item())         # id de clase
                    cls_name = model.names[cls_id]          # nombre de la clase
                    #print(cls_id, cls_name)

                boxes = r.boxes.xyxy.cpu().numpy()

                for i in range(len(r.boxes)):
                  cls_id = int(r.boxes[i].cls[0].item())         # id de clase
                  cls_name = model.names[cls_id]

                  if cls_name in 'car truck bus motorcycle'.split():
                    x1,y1,x2,y2 = boxes[i][:4]
                    draw.rectangle([x1,y1,x2,y2], outline='red', width=3)

          display(img)
          vehicle_count += 1

---

## 6) Conclusiones

- Las CNNs son efectivas para extraer características jerárquicas de imágenes.
- Para detección en escenas urbanas, usar detectores (YOLO, Faster-RCNN) permite localizar múltiples instancias; entrenar clasificadores con crops es útil pedagógicamente.
- En un taller: usar subconjuntos y modelos pequeños (`yolov8n`) para tiempos razonables.

---

## Ejercicios

1. Modifica el pipeline de crops para añadir `person` y entrena un clasificador ternario (vehicle, person, background).



¡Eso es todo amigos!